In [3]:
def ex_euclid(a,b):
    x0, x1 = 1, 0
    y0, y1 = 0, 1
    while b!=0:
        q, r = divmod(a, b) # r = a0 % a1, q = (a0 – r)//a1 でも可
        x0, x1 = x1, x0 - q * x1
        y0, y1 = y1, y0 - q * y1
        a, b = b, r
    return [a, x0, y0]

In [4]:
def mod_binary(g,k,n):
    # out: g^k mod n
    bk = bin(k)[2:]
    y = 1
    for i in bk:
        y = y * y % n
        if i == '1':
            y = y * g % n
    return y

In [5]:
def gcd(a,b):
    return ex_euclid(a,b)[0]

In [6]:
def lcm(a,b):
    return int(a*b/gcd(a,b))

In [7]:
def inv(a,n):
    return ex_euclid(a,n)[1] % n

In [8]:
def rsaSignGenKey(p, q, e):
    n  = p*q
    lambda_n = lcm(p-1, q-1)
    if gcd(e,lambda_n) != 1:
        return 0
    d = inv(e, lambda_n)
    return(n, e, d)

In [9]:
import hashlib
import binascii
def shake128(m, h_size):
    mhash = hashlib.shake_128(m.encode()).hexdigest(h_size)
    return int(mhash, 16)

In [59]:
m = "私はサイモン・シンの「暗号解読」の本が好きで、RSAの演習で非常に楽しいと感じました"
n = 132639128328156874302341968448627869114681769925253308356386508014896474072062362479952607784543134302534592007855554898302262711335875317907787788175302032187091773786443929734402942858254023783150409433342703442152907413461389601036781602744581302475935905417201036386878337159526581802689674768037898370557
e = 1009483867

256

In [56]:
import binascii

m_bytes = binascii.hexlify(m.encode())
m_int = int.from_bytes(m_bytes, "big")
m_int

37549337890029230658809638615972162136310897930934502970348791899520709895581373974261458393447991293975227436561923941994612193647716393645906380076245468029894559213260920260774815677212877545883149300737938553368974221241670503035940401551002093375189680679030521989741073365813723170368660367120701383375510075550956405535113309462357725758422135796750889204847202656004307989803737138997619340331105049451398913889560203061290584551152016747411714421535727506336517329672121412518730828613333930028511422869443739784339522451250937674888727768042101952460143400685780679014

In [53]:
def rsaEnc(m, n, e):
    return mod_binary(m, e, n)

In [54]:
rsaEnc(m_int, n, e)

65082753259677614840302317440102255033606812498334575278965996634325118852073894642910083643506412342896151247738462299807150252022983185188035617817136214839479014114624638301090892013180041728629551538457302786877865860870923229844268868526818857735561390997192946229795694765539119238816634935404597222417

In [77]:
def rsaSignGen(m, n, d):
    mhash = shake128(m, 127)
    return mod_binary(mhash, d, n)

In [92]:
sigma = rsaSignGen(m,n,d)
sigma

46455101874045784090816314737552982315724020408958029746002584700961383994513064039302147371562213041406512404431456062563010981263261223178829219168731802957512472217387342779964208706882025215330054278468107651153755844596184353960319022362126445535057182554440636941620621008841503982348875316194726445900

In [91]:
def  rsaSignVerify(m, sigma, n, e):
    mhash = shake128(m, 127)
    return mhash == mod_binary(sigma,e,n)

In [93]:
 rsaSignVerify(m, sigma, n, e)

True

In [101]:
import time
process_times = []
for i in range(1000):
    start = time.perf_counter()
    rsaSignGen(m, n, d)
    end = time.perf_counter()
    process_times.append(end-start)
process_times[0:10]

[0.010976374998790561,
 0.010017708000304992,
 0.00533233300120628,
 0.0053571660009765765,
 0.007964041999002802,
 0.006333999999696971,
 0.009108624999498716,
 0.006772750000891392,
 0.007504458000767045,
 0.00687879100041755]

In [102]:
import numpy as np

def mv(T):
    return np.mean(T), np.var(T)

res = mv(process_times)
print (res[0]*1000, res[1]*1e6)

5.0186282910181035 0.11315844375794387
